# Train Deep Q-Network

### 1. Install and import the Necessary Packages

In [ ]:
# !pip install gym
# !pip install gym[box2d]
# !pip install gym[classic_control]
# !pip -q install ./python

In [ ]:
from pathlib import Path
from collections import deque

import gym
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from unityagents import UnityEnvironment
from bananas_agent.dqn_agent import Agent

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [ ]:
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

In [ ]:
# get the default brain
BRAIN_NAME = env.brain_names[0]
brain = env.brains[BRAIN_NAME]

env_info = env.reset(train_mode=True)[BRAIN_NAME]
state = env_info.vector_observations[0]
ACTION_SIZE = brain.vector_action_space_size
STATE_SIZE = len(state)

In [ ]:
agent = Agent(state_size=STATE_SIZE, action_size=ACTION_SIZE)

### 3. Train the Agent with DQN

In [ ]:
# setup train parameters
n_episodes = 2000
max_steps = 2000
eps_max = 1.
eps_min = 0.01
min_expected_score = -3
target_score = 13
warm_start = False

In [ ]:
# warm start
CHECKPOINT_PATH = Path('checkpoints/checkpoint.pth')
if warm_start and CHECKPOINT_PATH.is_file():
    print("Using warm start!")
    agent.qnetwork_local.load_state_dict(torch.load(CHECKPOINT_PATH))
    agent.update_target_model()

In [ ]:
def run_episode(env, epsilon, max_steps):
    """ Run a single episode. """
    state = env.reset(train_mode=True)[BRAIN_NAME].vector_observations[0]
    score = 0
    for t in range(max_steps):
        action = agent.act(state, epsilon)
        env_info = env.step(action)[BRAIN_NAME]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    return score

def epsilon_interpolator(current_score):
    """ Update epsilon with a log interpolation based on average score. """
    eps_max_log, eps_min_log = np.log(eps_max), np.log(eps_min)
    return np.e ** np.interp(
        current_score,
        xp=[min_expected_score, target_score],
        fp=[eps_max_log, eps_min_log],
        left=eps_max_log,
        right=eps_min_log,
    )

scores = []
# compute average score based on last 100 scores
scores_window = deque(maxlen=100)

In [ ]:
epsilon = eps_max
for i_episode in range(1, n_episodes + 1):
    score = run_episode(env, epsilon, max_steps)
    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    average_score = np.mean(scores_window)
    epsilon = epsilon_interpolator(average_score)
    print('\rEpisode {}\tAverage Score: {:.2f}\tEpsilon: {:.2f}'.format(i_episode, average_score, epsilon), end="")
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}\tEpsilon: {:.2f}'.format(i_episode, average_score, epsilon))
    if average_score >= target_score:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        break

In [ ]:
# plot the score evolution
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
fig.savefig('score_evolution.png')

In [ ]:
torch.save(agent.qnetwork_local.state_dict(), CHECKPOINT_PATH)